In [1]:
import sys
import os
import shutil
from scipy.io.wavfile import read, write
from pydub import AudioSegment
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
os.chdir("/Users/venkatakrishnanvk/Desktop/Music Technology/Datasets/carnatic")

In [4]:
dest_folder = "/Users/venkatakrishnanvk/Desktop/Music Technology/Datasets/carnatic/violin_solo_dataset/vocal/orig_data"

# For copying of all multirack vocals

In [4]:
# for path in os.scandir(dest_folder):
#     if os.path.isfile(path.path):
#         os.remove(path.path)

# for path, dirs, files in os.walk(os.getcwd()):
#     for file in files:
#         if file.endswith("multitrack-vocal.mp3"):
#             src = path+"/"+file
#             if os.path.exists(dest_folder+file):
#                 continue
#             shutil.copy(src, dest_folder+file)

# Chopping to 10 minute audios

In [5]:
for path in os.scandir(dest_folder):
    if os.path.isfile(path.path):
        print(path.name[:-4])

Rama Neepai.multitrack-vocal2
Tulasi Bilva.multitrack-vocal
Shloka Sri Ramachandra Shrita Parijata.multitrack-vocal
Paramatmudu.multitrack-vocal1
Shri Kamakshi Shritajana.multitrack-vocal1
Rama Rama Guna Seema.multitrack-vocal5
RTP Andholika.multitrack-vocal3
Sri Guru Na Palitosmi.multitrack-vocal
Karuna Nidhi Illalo.multitrack-vocal1
Chandrashekharam Sada Bhajeham.multitrack-vocal
RTP Andholika.multitrack-vocal2
Prathi Vaaram Vaaram.multitrack-vocal
Rama Rama Guna Seema.multitrack-vocal4
Merusamana.multitrack-vocal1
Eppadi Padinaro.multitrack-vocal
Koluvamaregatha.multitrack-vocal6
Chakkani Raja.multitrack-vocal6
Chenduril Nindradum Kanda.multitrack-vocal
Rama Neepai.multitrack-vocal1
Chakkani Raja.multitrack-vocal4
Koluvamaregatha.multitrack-vocal4
Paramatmudu.multitrack-vocal2
Janakipathe Jaya Karunya Jaladhe.multitrack-vocal5
Rama Rama Guna Seema.multitrack-vocal6
Shri Kamakshi Shritajana.multitrack-vocal2
Mati Matiki.multitrack-vocal4
Karuna Nidhi Illalo.multitrack-vocal2
Sarasuda

In [6]:
# for path in os.scandir(dest_folder):
#     if os.path.isfile(path.path):
#         # fs, audio = read(path.path)
#         audio_data = AudioSegment.from_file(path.path, "mp3")
#         fs = int(audio_data.frame_rate)
#         if len(audio_data)/60000 > 10: # 10 minute chunks
#             pointer = 0
#             while True:
#                 new_audio = audio_data[pointer*600000:(pointer+1)*600000]
#                 new_audio.export(dest_folder+path.name[:-4]+str(pointer+1)+".mp3", format = "mp3")
#                 pointer+=1
#                 if len(new_audio) != 600000:
#                     os.remove(path.path)
#                     break

In [7]:
os.chdir(dest_folder)

# Splitting source separated audio into 3 second chunks

In [28]:
directory = "/Users/venkatakrishnanvk/Desktop/Music Technology/Datasets/carnatic/violin_solo_dataset/vocal/vocalists_no_ss"

In [29]:
count = len(os.listdir(directory)) + 1
count

15570

In [30]:
count = 581

In [31]:
def pydub_to_np(audio: AudioSegment) -> (np.ndarray, int):
    """
    Converts pydub audio segment into np.float32 of shape [duration_in_seconds*sample_rate, channels],
    where each value is in range [-1.0, 1.0]. 
    Returns tuple (audio_np_array, sample_rate).
    """
    return np.array(audio.get_array_of_samples(), dtype=np.float32).reshape((-1, audio.channels)) / (
            1 << (8 * audio.sample_width - 1)), audio.frame_rate


## For Source Separated

In [32]:
# for path, dirs, files in os.walk(dest_folder+"ss_data"):
#     for file in files:
#         if file == "vocals.mp3":
#             audio_data = AudioSegment.from_file(path+"/"+file, "mp3")
#             fs = int(audio_data.frame_rate)
#             pointer = 0
#             while True:
#                 new_audio = audio_data[pointer*3000:(pointer+1)*3000] # 3 s of audio
#                 new_audio_np = pydub_to_np(new_audio)[0]
#                 if not np.isnan(np.mean(np.abs(new_audio_np.T[0]))) and np.mean(np.abs(new_audio_np.T[0])) > 0.0001:
#                     pointer+=1
#                     write("violin_solo_dataset/vocal/vocalists/vocal"+str(count)+".wav", 44100, new_audio_np.T[0])
#                     # new_audio.export("violin_solo_dataset/vocal/vocalists/vocal"+str(count)+".wav", format = "wav")
#                     # sys.exit()
#                     count+=1
#                     pointer+=1
#                 else:
#                     pointer+=1
#                 if len(new_audio)<3000:
#                     break

## For Original

In [33]:
for path, dirs, files in os.walk(dest_folder):
    for file in files:
        if file.endswith(".mp3"):
            audio_data = AudioSegment.from_file(path+"/"+file, "mp3")
            fs = int(audio_data.frame_rate)
            pointer = 0
            while True:
                new_audio = audio_data[pointer*3000:(pointer+1)*3000] # 3 s of audio
                new_audio_np = pydub_to_np(new_audio)[0]
                if not np.isnan(np.mean(np.abs(new_audio_np.T[0]))) and np.mean(np.abs(new_audio_np.T[0])) > 0.0001:
                    new_audio_np = new_audio_np/np.max(new_audio_np)
                    pointer+=1
                    write("/Users/venkatakrishnanvk/Desktop/Music Technology/Datasets/carnatic/violin_solo_dataset/vocal/vocalists_no_ss/vocal"+str(count)+".wav", 44100, new_audio_np.T[0])
                    count+=1
                    pointer+=1
                else:
                    pointer+=1
                if len(new_audio)<3000:
                    break